In [1]:
import pandas as pd
#from SPARQLWrapper import SPARQLWrapper, JSON
#import ipython_sparql
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF, CSV, TSV
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from IPython.display import clear_output
import time
import csv
import joblib

In [2]:
words_to_scrap = pd.DataFrame(pd.read_csv('main_data.csv'))
data=words_to_scrap['clean_presentation_trad']

In [3]:
word ='\\u200b\\u200brue'
re.sub('\\\\','',word)

'u200bu200brue'

In [4]:
data=[re.sub('\[','',sent) for sent in data]
data=[re.sub('\]','',sent) for sent in data]
data=[re.sub('\"','',sent) for sent in data]
data=[re.sub('''\\\\n''','',sent) for sent in data]
data=[re.sub('''\\\\''','',sent) for sent in data]
data=[sent.split(',') for sent in data]
data=[[re.sub("\'",'',sent) for sent in array] for array in data]
data=[[sent if sent!=''else None for sent in array] for array in data]
data=[[re.sub(" ",'',sent) for sent in array] for array in data]
data=[[re.sub("-",'',sent) for sent in array] for array in data]

In [5]:
data[0][2].capitalize()

'Production'

In [6]:
len(data)

209

In [7]:
NewData=[]
for array in data:
    Ligne=[]
    for word in array:
        Ligne.append(word.capitalize())
    NewData.append(Ligne)

In [8]:
data=NewData

Scrapping DB pédia et rangement en liste de liste par mots

In [9]:
fail=0
subject=[]
start_time=time.time()
resultats=[]
i=0
for array in data:
    i+=1
    j=0
    for word in array:
        j+=1
        resultats=[]
        print('Site : '+str(i/len(data)*100)[:4]+'%')
        print('mot / site : '+str(j/len(array)*100)[:4]+'%')
        print('Site(s) restant : '+str(len(data)-i))
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT  ?subject
        WHERE { 
        <http://dbpedia.org/resource/"""+word+"""> dct:subject ?subject
        }
        """)
        sparql.setReturnFormat(JSON)
        #results.append(sparql.query().convert())
        results=sparql.query().convert()
        if (results["results"]["bindings"] != []):
            for n in range (len(results["results"]["bindings"])):
                resultats.append(results["results"]["bindings"][n]["subject"]["value"])
            #results = results["results"]["bindings"][0]
            #subject.append(results["subject"]["value"])
            subject.append(resultats)
        else:
            fail+=1
            if (fail%100 == 0):
                print("nombre de données manquantes :", fail)
                subject.append(resultats)
        clear_output(wait=True)
        

interval = time.time() - start_time  
print('Temps total : '+str(interval))

Temps total : 1203.2850568294525


In [10]:
len(subject)

5278

In [11]:
print ( "Nombre d'erreur",fail)

Nombre d'erreur 10097


Scrapping DB pédia et rangement en liste de liste par Site

In [ ]:
fail=0
subject=[]
start_time=time.time()
resultats=[]
i=0
for array in data:
    i+=1
    j=0
    resultats=[]
    for word in array:
        j+=1
        print('Site : '+str(i/len(data)*100)[:4]+'%')
        print('mot / site : '+str(j/len(array)*100)[:4]+'%')
        print('Site(s) restant : '+str(len(data)-i))
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery("""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT  ?subject
        WHERE { 
        <http://dbpedia.org/resource/"""+word+"""> dct:subject ?subject
        }
        """)
        sparql.setReturnFormat(JSON)
        #results.append(sparql.query().convert())
        results=sparql.query().convert()
        if (results["results"]["bindings"] != []):
            for n in range (len(results["results"]["bindings"])):
                resultats.append(results["results"]["bindings"][n]["subject"]["value"])
            #results = results["results"]["bindings"][0]
            #subject.append(results["subject"]["value"])
        else:
            fail+=1
            if (fail%100 == 0):
                print("nombre de données manquantes :", fail)
        clear_output(wait=True)
    subject.append(resultats)
        

interval = time.time() - start_time  
print('Temps total : '+str(interval))

Site : 25.3%
mot / site : 59.3%
Site(s) restant : 156


Resultat Classement liste de liste par Site

In [91]:
len(subject)

209

In [84]:
subject

[['http://dbpedia.org/resource/Category:Office_buildings',
  'http://dbpedia.org/resource/Category:Rooms',
  'http://dbpedia.org/resource/Category:Office_administration'],
 ['http://dbpedia.org/resource/Category:Structure',
  'http://dbpedia.org/resource/Category:Systems'],
 ['http://dbpedia.org/resource/Category:Aesthetics',
  'http://dbpedia.org/resource/Category:Arts',
  'http://dbpedia.org/resource/Category:Visual_arts'],
 ['http://dbpedia.org/resource/Category:Ballot_measures',
  'http://dbpedia.org/resource/Category:Direct_democracy',
  'http://dbpedia.org/resource/Category:Right_to_petition',
  'http://dbpedia.org/resource/Category:Sources_of_law',
  'http://dbpedia.org/resource/Category:Initiatives'],
 ['http://dbpedia.org/resource/Category:France',
  'http://dbpedia.org/resource/Category:Countries_in_Europe',
  'http://dbpedia.org/resource/Category:French-speaking_countries_and_territories',
  'http://dbpedia.org/resource/Category:G20_nations',
  'http://dbpedia.org/resource/C

In [100]:
donnée= joblib.dump(subject, "dataScrapParSite.job" )

In [96]:
words_to_scrap['DBPedia'] = subject

In [98]:
words_to_scrap

,Unnamed: 0,id,nom,presentation,presentation_en,clean_presentation_trad,DBPedia
0,0,6,Le Commissariat,Le Commissariat est une structure de productio...,The Office is a structure of production of con...,"['Office', 'structure', 'production', 'contemp...",[http://dbpedia.org/resource/Category:Office_b...
1,1,198,/ 77,-English below- \n/ 77 est un collectif d'art...,-English below-\n/ 77 is a collective of artis...,"['below-', 'collective', 'artists', 'composed'...",[http://dbpedia.org/resource/Category:Collecti...
2,2,4,0gms,"0gms is an Artist-Run-Space based in Sofia, Bu...",NaN,"['gms', 'Artist', 'Run', 'Space', 'based', 'So...",[http://dbpedia.org/resource/Category:Aestheti...
3,3,87,Lieu d'exposition en voie de déplacement (LEVD),Lieu d’exposition en voie de déplacement (LEVD...,Instead of moving lane exposure (ESA) invests ...,"['Instead', 'moving', 'lane', 'exposure', 'ESA...",[http://dbpedia.org/resource/Category:Road_inf...
4,4,88,mezcla,\nAssociation au service de la promotion et d...,\nAssociation to promote service and mediation...,"['Association', 'promote', 'service', 'mediati...",[http://dbpedia.org/resource/Category:Mediatio...
5,5,90,BLINDSIDE,BLINDSIDE is an independent contemporary arts ...,NaN,"['BLINDSIDE', 'independent', 'contemporary', '...",[http://dbpedia.org/resource/Category:Visual_a...
6,6,39,H+,"La Galerie H+ est une galerie lyonnaise, créée...","The H + Gallery is a gallery Lyon, created on ...","['Gallery', 'gallery', 'Lyon', 'created', 'spe...","[http://dbpedia.org/resource/Category:Lyon, ht..."
7,7,169,L’Assaut de la menuiserie,Dédiée à la jeune création contemporaine sous ...,Dedicated to young contemporary creation in al...,"['Dedicated', 'young', 'contemporary', 'creati...",[http://dbpedia.org/resource/Category:Carpentr...
8,8,91,KINGS Artist Run,\nKings Artist Run is a non-for-profit gallery...,NaN,"['Kings', 'Artist', 'Run', 'non', 'profit', 'g...",[http://dbpedia.org/resource/Category:Aestheti...
9,9,170,Quellkollektiv,L'association Quellkollektiv émerge de l'assoc...,The combination Quellkollektiv emerges from th...,"['combination', 'Quellkollektiv', 'emerges', '...",[http://dbpedia.org/resource/Category:Combinat...


In [99]:
words_to_scrap.to_csv('dataScrape.csv')